In [58]:
##
##2021.03.27 #백재원
##

import pandas as pd #campus_tel_20210310
import numpy as np
import pymysql
import logging
#import matplotlib
from matplotlib import font_manager

font_location  = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname = font_location).get_name()
logger = logging.getLogger()

conn = pymysql.connect(
    host='114.71.137.109',
    user='202147018',
    password='bjy969920',
    database='Numberbot',
    charset='utf8'
)
cur = conn.cursor()

def cut(Data):
    rows = int(Data.shape[1] / 2)
    name = '성명'
    number = '번호'
    TotalData = pd.DataFrame({'성명' : Data[name], '번호' : Data[number]})
    for i in np.arange(1,rows):
        name = name + '.' + str(i)
        number = number + '.' + str(i)
        Temp = pd.DataFrame({'성명' : Data[name], '번호' : Data[number]})
        TotalData = TotalData.append(Temp)
        name = '성명'
        number = '번호'    
    return TotalData

def SumData(Data):
    name = '성명'
    number = '번호'
    
    in_name = ""
    in_position = ""
    
    in_group = ""
    in_bulid = ""
    
    TotalData = pd.DataFrame()
    col = Data[name][0]
    ckcnt = 0
    for i in np.arange(0,Data[name].count() + 21):
       
    #이름 " "이 아닌 이름(ㅁㅁ) 붙이있는 애들 수정 필요....
        if(type(Data[name][i]) == str):
            nmcnt = Data[name][i].count(" ")
            #sp = Data[name][i].split(" ")
            sp = Data[name][i].split(" ")
        
        if(type(Data[number][i]) == float and type(Data[name][i])  == str):
                col = Data[name][i].split("(")
                gpcnt = Data[name][i].count("(")
                continue
        
        if(gpcnt > 0 or type(gpcnt) != int):
            in_group = col[0]
            in_bulid = col[1]
        elif(nmcnt < 1):
            in_group = col[0]
            in_bulid = col[0]
        in_bulid = in_bulid.replace(')','')

        if(nmcnt > 0 or type(nmcnt) != int):
            in_name = sp[1]
            in_position = sp[0]
        elif(nmcnt < 1):
            in_name = sp[0]
            in_position = sp[0]
        in_position = in_position.replace('(','')
        in_position = in_position.replace(')','')
        
        TotalData = TotalData.append({'성명' : in_name, '번호' : Data[number][i], '그룹' : in_group, '위치' : in_bulid, '직책' : in_position}, ignore_index=True)
            #index = index + 1
    return TotalData

def SQLINSERT(Data):
    title = Data['그룹'][0]
    chapter_no = 1
    group_no = 1
    
    for i in np.arange(0,Data['성명'].count() + 21):
        if(title != Data['그룹'][i]):
            title = Data['그룹'][i]
            chapter_no = chapter_no + 1
            group_no = 1
        chapter_title = TotalData["그룹"][i]
        group_title = TotalData["성명"][i]
        content = str(TotalData["번호"][i])
        
        sql = 'insert into Number_dictionary(doc_class, chapter_no, chapter_title, group_no, group_title, content) values(1, %s, %s, %s, %s, %s)'
        cur.execute(sql,(chapter_no, chapter_title, group_no, group_title, content))
        group_no = group_no + 1

Data = pd.read_excel("C:/Temp/campus_tel_20210310.xlsx",  skiprows=2, index_col=False)

Data = cut(Data)

Data = Data.reset_index()
Data = Data.drop("index", axis=1)
print(Data)
TotalData = SumData(Data)

#TotalData.loc['총장실', '그룹'] = "뭐야"

#try:
 #SQLINSERT(TotalData)
#except Exception as e:
#    logger.error(e)
#    logger.exception(e)
#    raise
#finally:
#    conn.commit()
#    cur.close()
#    conn.close()


excel_writer = pd.ExcelWriter('C:\\Temp\\tel_output.xlsx', engine='xlsxwriter')
TotalData.to_excel(excel_writer, index = False)
excel_writer.save()


TotalData.head(200)


            성명        번호
0          총장실       NaN
1       총장 진인주      2000
2         (직통)  865-6991
3     (비서) 전시은      2001
4          FAX  868-3408
..         ...       ...
310        강사실       NaN
311   3호관 503호      2638
312   4호관 308호      2631
313   5호관 308호      2430
314  11호관 509호      2435

[315 rows x 2 columns]


,그룹,번호,성명,위치,직책
0,,2000,진인주,,총장
1,총장실,865-6991,(직통),총장실,직통
2,총장실,2001,전시은,총장실,비서
3,총장실,868-3408,FAX,총장실,FAX
4,총장실,2002,김용진,총장실,처장
...,...,...,...,...,...
195,전산정보원,2105,조성일,7-409,보안/IPT
196,전산정보원,2106,김대원,7-409,네트워크
197,전산정보원,2111,민원안내,7-409,민원안내
198,전산정보원,868-3508,FAX,7-409,FAX


pip install PyMySQL